In [10]:
# Import The Liabraries
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats

In [11]:
# Load The Tickers
tickers = pd.read_csv('top_50_indian_stocks.csv')
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


In [12]:
# Weighted Scoring Model For Finding The Final Score
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
        
    ## we have got our df with dividend information
    
    numeric_cols = [
          "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    

    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df


In [13]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,35.0,5.0,9.779999,44.0,2.4,0.028818,0.029851,0.066549,0.042411,0.079428
1,TCS.NS,169.0,60.0,43.220000,128.0,-1.7,0.157541,0.440299,0.323227,0.136161,0.074948
2,HDFCBANK.NS,115.0,22.0,21.110001,67.0,6.0,0.105668,0.156716,0.153516,0.068080,0.083361
3,INFY.NS,396.0,63.0,64.440000,212.0,-12.7,0.375600,0.462687,0.486107,0.229911,0.062930
4,ICICIBANK.NS,70.0,10.0,14.060001,99.0,16.1,0.062440,0.067164,0.099401,0.103795,0.094395
5,HINDUNILVR.NS,204.0,48.0,94.879997,150.0,-3.8,0.191162,0.350746,0.719757,0.160714,0.072654
6,SBIN.NS,199.0,15.9,15.760000,139.0,-8.3,0.186359,0.111194,0.112450,0.148438,0.067737
7,BAJFINANCE.NS,48.0,44.0,13.420000,24.0,17.0,0.041306,0.320896,0.094489,0.020089,0.095379
8,BHARTIARTL.NS,44.0,8.0,14.280000,80.0,423.0,0.037464,0.052239,0.101090,0.082589,0.538949
9,ITC.NS,326.0,14.0,85.350000,407.0,-7.5,0.308357,0.097015,0.646607,0.447545,0.068611


In [14]:
weights = {
        "Dividend Yield(%) Normalised": 0.3,
        "Dividend Rate Normalised": 0.2,
        "Payout Ratio(%) Normalised": 0.2,
        "5 Year Avg Dividend Yield(%) Normalised": 0.2,
        "Earning Growth(%) Normalised": 0.1
    }

dividend_df["Dividend Score"] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
0,RELIANCE.NS,35.0,5.0,9.779999,44.0,2.4,0.028818,0.029851,0.066549,0.042411,0.079428,0.044350
1,TCS.NS,169.0,60.0,43.220000,128.0,-1.7,0.157541,0.440299,0.323227,0.136161,0.074948,0.234694
2,HDFCBANK.NS,115.0,22.0,21.110001,67.0,6.0,0.105668,0.156716,0.153516,0.068080,0.083361,0.115699
3,INFY.NS,396.0,63.0,64.440000,212.0,-12.7,0.375600,0.462687,0.486107,0.229911,0.062930,0.354714
4,ICICIBANK.NS,70.0,10.0,14.060001,99.0,16.1,0.062440,0.067164,0.099401,0.103795,0.094395,0.082244
5,HINDUNILVR.NS,204.0,48.0,94.879997,150.0,-3.8,0.191162,0.350746,0.719757,0.160714,0.072654,0.310858
6,SBIN.NS,199.0,15.9,15.760000,139.0,-8.3,0.186359,0.111194,0.112450,0.148438,0.067737,0.137098
7,BAJFINANCE.NS,48.0,44.0,13.420000,24.0,17.0,0.041306,0.320896,0.094489,0.020089,0.095379,0.109025
8,BHARTIARTL.NS,44.0,8.0,14.280000,80.0,423.0,0.037464,0.052239,0.101090,0.082589,0.538949,0.112318
9,ITC.NS,326.0,14.0,85.350000,407.0,-7.5,0.308357,0.097015,0.646607,0.447545,0.068611,0.337602


In [15]:
# Sort According To Dividend Score
dividend_df = dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
45,IOC.NS,1046.0,15.0,70.919997,732.0,57.8,1.000000,0.104478,0.535846,0.810268,0.139954,0.604114
30,COALINDIA.NS,784.0,31.6,45.930000,902.0,-29.2,0.748319,0.228358,0.344028,1.000000,0.044903,0.543463
42,HEROMOTOCO.NS,320.0,130.0,64.019996,317.0,24.2,0.302594,0.962687,0.482883,0.347098,0.103245,0.459636
13,HCLTECH.NS,440.0,72.0,84.260005,301.0,1.6,0.417867,0.529851,0.638241,0.329241,0.078553,0.432682
20,ONGC.NS,711.0,17.5,40.310000,524.0,-16.8,0.678194,0.123134,0.300890,0.578125,0.058451,0.409733
22,POWERGRID.NS,507.0,15.0,70.519996,512.0,-4.2,0.482229,0.104478,0.532776,0.564732,0.072217,0.392287
31,BPCL.NS,496.0,15.5,49.660000,569.0,-8.4,0.471662,0.108209,0.372659,0.628348,0.067628,0.370105
3,INFY.NS,396.0,63.0,64.440000,212.0,-12.7,0.375600,0.462687,0.486107,0.229911,0.062930,0.354714
41,TATASTEEL.NS,232.0,3.6,131.390000,284.0,112.2,0.218060,0.019403,1.000000,0.310268,0.199388,0.351291
28,TECHM.NS,281.0,45.0,89.750000,225.0,76.4,0.265130,0.328358,0.680381,0.244420,0.160275,0.346198
